In [53]:
from openai import OpenAI
import pandas as pd
import numpy as np
import os

client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY']
)

df = pd.read_csv('words.csv')
df

,text
0,red
1,potatoes
2,soda
3,cheese
4,water
5,blue
6,crispy
7,hamburger
8,coffee
9,green


In [55]:
cappuccino = client.embeddings.create(input="cappuccino", model="text-embedding-ada-002")

In [67]:
def get_embedding(word):
    result = client.embeddings.create(input=word, model="text-embedding-ada-002")
    return result.data[0].embedding

In [69]:
cappuccino.data[0].embedding

[-0.01667233742773533,
 -0.01890314184129238,
 0.010240833275020123,
 -0.05353933572769165,
 -0.003952831029891968,
 -0.004298540763556957,
 -0.003086926182731986,
 -0.009725529700517654,
 0.004716001451015472,
 -0.0277872271835804,
 0.014180618338286877,
 0.028752606362104416,
 -0.016320103779435158,
 -0.01656797155737877,
 -0.016450561583042145,
 -0.012869530357420444,
 0.0371800921857357,
 0.01043651718646288,
 0.03018762730062008,
 -0.013163058087229729,
 -0.015602593310177326,
 0.03123127855360508,
 -0.003414698177948594,
 -0.020025068894028664,
 -0.014128435403108597,
 -0.007377313449978828,
 -0.002327017253264785,
 -0.01886400580406189,
 0.0008463363628834486,
 -0.0064445496536791325,
 0.020546894520521164,
 -0.007892616093158722,
 -0.02859605848789215,
 -0.010234310291707516,
 -0.014754626899957657,
 -0.024395359680056572,
 -0.00521173607558012,
 -0.015459091402590275,
 0.009321114979684353,
 -0.003072249935939908,
 0.008101346902549267,
 -0.003793021896854043,
 -0.011838924139

In [72]:
df["embedding"] = df["text"].apply(lambda x: get_embedding(x))
df.to_csv("word_embeddings.csv", index=False)

In [73]:
import ast

def list_of_float64(value):
    return np.array(ast.literal_eval(value), dtype=np.float64)
df = pd.read_csv("word_embeddings.csv", converters={"embedding": list_of_float64})
df


,text,embedding
0,red,"[-0.00010086999100167304, -0.02487374469637870..."
1,potatoes,"[0.005065467208623886, -0.031116440892219543, ..."
2,soda,"[0.025743870064616203, -0.007405862212181091, ..."
3,cheese,"[-0.003227480687201023, -0.008714198134839535,..."
4,water,"[0.019110165536403656, -0.012494578957557678, ..."
5,blue,"[0.005463808309286833, -0.007340267766267061, ..."
6,crispy,"[-0.000983826699666679, -0.0053982213139534, -..."
7,hamburger,"[-0.013203660026192665, -0.0018609294202178717..."
8,coffee,"[-0.000712389184627682, -0.0196177177131176, 0..."
9,green,"[0.015380158089101315, -0.011085166595876217, ..."


In [74]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [75]:
df["similarities"] = df["embedding"].apply(lambda x: cosine_similarity(x, cappuccino.data[0].embedding))
res = df.sort_values("similarities", ascending=False)
res

,text,embedding,similarities
19,espresso,"[-0.022567009553313255, -0.012796901166439056,...",0.916284
15,latte,"[-0.015810398384928703, -0.0038977975491434336...",0.892797
22,mocha,"[-0.012522608041763306, -0.026235325261950493,...",0.877275
8,coffee,"[-0.000712389184627682, -0.0196177177131176, 0...",0.869270
10,milk,"[0.0009192764409817755, -0.019397875294089317,...",0.846036
13,chocolate,"[0.0015385933220386505, -0.01302750501781702, ...",0.822187
3,cheese,"[-0.003227480687201023, -0.008714198134839535,...",0.810073
20,cheesecake,"[0.010881966911256313, -0.012525497004389763, ...",0.806436
7,hamburger,"[-0.013203660026192665, -0.0018609294202178717...",0.801807
18,cheeseburger,"[-0.01848696544766426, 0.005220311228185892, 0...",0.799971
